In [14]:
import chromadb

client = chromadb.PersistentClient(path="./my_notes_db")
collection = client.get_or_create_collection(name="personal_notes")

notes = [
    "Python is a versatile programming language used for AI.",
    "The grocery list includes milk, bread, and eggs.",
    "Remember to finish the vector database project by Friday."
]

collection.add(
    documents=notes,
    ids=["note1", "note2", "note3"]
)

query = "What food do I need to buy?"
results = collection.query(query_texts=[query], n_results=1)

results

{'ids': [['note2']],
 'embeddings': None,
 'documents': [['The grocery list includes milk, bread, and eggs.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[1.0297448635101318]]}

In [ ]:
import os
import chromadb
from dotenv import load_dotenv
from google import genai

load_dotenv()

client_gemini = genai.Client() 
MODEL_ID = "gemini-2.5-flash"

client_db = chromadb.PersistentClient(path="./my_notes_db")
collection = client_db.get_or_create_collection(name="personal_notes")

def ask_gemini_with_rag(user_query):
    results = collection.query(query_texts=[user_query], n_results=1)
    
    if not results['documents'] or not results['documents'][0]:
        return "I couldn't find any relevant notes to answer that."
        
    retrieved_context = results['documents'][0][0]

    prompt = f"""
    You are a helpful assistant. Use the following piece of retrieved context 
    to answer the user's question. If the answer isn't in the context, say you don't know.
    
    Context: {retrieved_context}
    
    Question: {user_query}
    
    Answer:
    """
    response = client_gemini.models.generate_content(
        model=MODEL_ID,
        contents=prompt
    )
    
    return response.text

query = "What food do I need to buy?"
print(f"AI Response: {ask_gemini_with_rag(query)}")

AI Response: You need to buy milk, bread, and eggs.
